**Generating Elasticsearch Top-k Results for Re-ranking**

In [1]:
!pip3 install elasticsearch

In [2]:
!pip3 install elasticsearch-dsl

In [3]:
from elasticsearch import Elasticsearch

import sys
sys.path.insert(0, '../../../BERT-FAQ/')

# import required classes and functions
from reranker import ReRanker
from evaluation import get_relevance_label_df

# import utility functions
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs

In [4]:
# Generate list of test queries, relevance labels for ReRanker class
query_answer_pair_filepath = '../../../BERT-FAQ/data/FAQIR/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
test_queries = relevance_label_df[relevance_label_df['query_type'] == 'user_query'].question.unique()

In [5]:
# check first 5 occurences for test_queries
test_queries[:5]

array(['How to increase the efficiency of my car with respect to fuel expended?',
       'Is there a special way to drive to decreasse my fuel consumption?',
       'How to use less gas in my car?',
       'I want to save some money on gas. What can I do to make my car use less gas?',
       'Are there any simple tweaks one can do to a car to make it more fuel efficient?'],
      dtype=object)

In [6]:
# check relevance_label_df
relevance_label_df

,id,question,answer,query_type,label
0,1,"How to remove tree sap from car? Hi,I have som...",I have had good luck with commercial products ...,faq,1
1,2,How do I get a sticker off of a car window (p...,Goo Gone is a fantastic product (available at ...,faq,1
2,3,How to get rid of garbage disposal odor?,1. Inspect the black rubber cover that fits in...,faq,1
3,4,How to maintain a lawn mower?,"To properly maintain your lawnmover, you'll ne...",faq,1
4,5,How to fix vertical blinds? I can no longer re...,Click on the link and follow the steps.,faq,1
...,...,...,...,...,...
16319,16320,Cigarette smoke odor is so annoying. Some frie...,"Get a bucket, fold some cotton towels into it,...",user_query,1
16320,16321,I love smoking a pipe but the smell is quite d...,"Get a bucket, fold some cotton towels into it,...",user_query,1
16321,16322,What are the best tips for getting rid of the ...,"Get a bucket, fold some cotton towels into it,...",user_query,1
16322,16323,The weed smell is really hard to get rid of. D...,"Get a bucket, fold some cotton towels into it,...",user_query,1


In [7]:
# Define instance of ReRanker class
r = ReRanker(
    bert_model_path='', 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [8]:
# create output path to save Elasticsearch top-k results
output_path = "../../../BERT-FAQ/data/FAQIR/rank_results/unsupervised"
make_dirs(output_path)

In [9]:
# Select top-k value
top_k = 100

In [ ]:
# Get top-k Elasticsearch results 

es = Elasticsearch([{'host':'localhost','port':9200}], http_auth=('username', 'password')) 

es_query_by_question = r.get_es_topk_results(es=es, index='faqir', query_by=['question'], top_k=top_k)
es_query_by_answer = r.get_es_topk_results(es=es, index='faqir', query_by=['answer'], top_k=top_k)
es_query_by_question_answer = r.get_es_topk_results(es=es, index='faqir', query_by=['question', 'answer'], top_k=top_k)
es_query_by_question_answer_concat = r.get_es_topk_results(es=es, index='faqir', query_by=['question_answer'], top_k=top_k)

2021-03-02 12:47:08 - Generating ES top-k results ...
  0%|          | 0/1183 [00:00<?, ?it/s]2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.023s]
2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.017s]
2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.017s]
2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
  1%|          | 6/1183 [00:00<00:22, 51.95it/s]2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:08 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
2021-03-02 12:47:08 - POST http://local

2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
  7%|▋         | 84/1183 [00:01<00:19, 56.26it/s]2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.018s]
2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.019s]
  8%|▊         | 90/1183 [00:01<00:19, 55.20it/s]2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:10 - POST http://localhost:9200/faqir/_search [status:200 request:

2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.013s]
2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
 14%|█▍        | 170/1183 [00:02<00:15, 63.94it/s]2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.018s]
2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.017s]
2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:11 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:11 - POST http://localh

2021-03-02 12:47:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
2021-03-02 12:47:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.013s]
2021-03-02 12:47:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
 21%|██▏       | 254/1183 [00:04<00:14, 62.51it/s]2021-03-02 12:47:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
2021-03-02 12:47:13 - POST http://localhost:9200/faqir/_search [status:200 request:0.017s]
2021-03-02 12:47:13 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:13 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
2021-03-02 12:47:13 - POST http://localhost:9200/faqir/_search [status:200 request:0.013s]
2021-03-02 12:47:13 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:13 - POST http://localh

2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
 29%|██▊       | 338/1183 [00:05<00:13, 60.39it/s]2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.014s]
2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.016s]
2021-03-02 12:47:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.015s]
2021-03-02 12:47:14 - POST http://localh

...

2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.020s]
2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.020s]
 72%|███████▏  | 856/1183 [00:21<00:08, 40.07it/s]2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.018s]
2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 request:0.022s]
 73%|███████▎  | 861/1183 [00:22<00:07, 41.18it/s]2021-03-02 12:48:12 - POST http://localhost:9200/faqir/_search [status:200 reques

2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.026s]
2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.023s]
 79%|███████▉  | 936/1183 [00:23<00:05, 42.39it/s]2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.023s]
2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.025s]
2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.020s]
2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.024s]
 80%|███████▉  | 941/1183 [00:23<00:05, 41.89it/s]2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.025s]
2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 request:0.024s]
2021-03-02 12:48:14 - POST http://localhost:9200/faqir/_search [status:200 reques

 93%|█████████▎| 1096/1183 [00:27<00:02, 40.06it/s]2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.019s]
2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.026s]
2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.023s]
2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.023s]
 93%|█████████▎| 1101/1183 [00:27<00:02, 40.39it/s]2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.025s]
2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.023s]
2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:18 - POST http://localhost:9200/faqir/_search [status:200 requ

2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 request:0.022s]
2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 request:0.024s]
2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 request:0.024s]
2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 request:0.022s]
100%|█████████▉| 1181/1183 [00:29<00:00, 39.98it/s]2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 request:0.025s]
2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 request:0.022s]
100%|██████████| 1183/1183 [00:29<00:00, 39.46it/s]
2021-03-02 12:48:20 - Generating ES top-k results ...
  0%|          | 0/1183 [00:00<?, ?it/s]2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 request:0.022s]
2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 request:0.022s]
2021-03-02 12:48:20 - POST http://localhost:9200/faqir/_search [status:200 

  6%|▋         | 75/1183 [00:01<00:26, 41.31it/s]2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.025s]
2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.034s]
2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.026s]
2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.031s]
2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.031s]
  7%|▋         | 80/1183 [00:01<00:29, 37.69it/s]2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.023s]
2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.021s]
2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.025s]
2021-03-02 12:48:22 - POST http://localhost:9200/faqir/_search [status:200 request:0.024s]
  7%|▋         | 84/1183 [00:02<00:28, 38.08it/s]2021-03-02 12:48:22 - POST http://

In [ ]:
# Save Elasticsearch results to json files
dump_to_json(es_query_by_question, output_path + '/es_query_by_question.json')
dump_to_json(es_query_by_answer, output_path + '/es_query_by_answer.json')
dump_to_json(es_query_by_question_answer, output_path + '/es_query_by_question_answer.json')
dump_to_json(es_query_by_question_answer_concat, output_path + '/es_query_by_question_answer_concat.json')